In [4]:
import os

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/bhuvankatakam/Data-Science-Project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']  = "bhuvankatakam"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "e61dc15109606ab5331c5e385a5f81abc35e64a2"

In [6]:
import os

In [7]:
%pwd

'c:\\Users\\GMI-00108\\Downloads\\Data Science Project(CICD)\\Data-Science-Project\\research'

In [8]:
os.chdir('../')

In [9]:
%pwd

'c:\\Users\\GMI-00108\\Downloads\\Data Science Project(CICD)\\Data-Science-Project'

In [37]:
from dataclasses import dataclass
from pathlib import Path

In [38]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_path: Path
    model_path: Path
    evaluate_path: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [39]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [40]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH,schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.ELASTIC_NET
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_path = config.test_path,
            model_path= config.model_path,
            evaluate_path = config.evaluate_path,
            all_params = params,
            target_column = schema,
            mlflow_uri = "https://dagshub.com/bhuvankatakam/Data-Science-Project.mlflow"
            )
        return model_evaluation_config
    




In [41]:
import os
import mlflow
import mlflow.sklearn
import pandas as pd
from urllib.parse import urlparse
import numpy as np
import joblib



In [42]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error

In [47]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        mse = mean_absolute_error(actual,pred)
        rmse =  np.sqrt(mse)
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return mse,rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_path)
        target_column = list(self.config.target_column.keys())[0]
        test_x = test_data.drop([target_column],axis = 1)
        test_y = test_data[[target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            model = joblib.load(self.config.model_path)
            predicted_qualities = model.predict(test_x)
            (mse,rmse,mae,r2) = self.eval_metrics(test_y,predicted_qualities)
            scores = {"mse" : mse, "rmse": rmse, "mae": mae, "r2_score" : r2}
            save_json(path = Path(self.config.evaluate_path),data= scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("mse", mse)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model Registry does not work with file store
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model,"model", registered_model_name= "ELASTICNET")
            else:
                mlflow.sklear.log_model(model,"model")


    
        


In [48]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluate = ModelEvaluation(model_evaluation_config)
    model_evaluate.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-11 16:20:42,257: INFO: common: yaml file: config\config.yaml file loaded successfully]
[2025-02-11 16:20:42,260: INFO: common: yaml file: params.yaml file loaded successfully]
[2025-02-11 16:20:42,265: INFO: common: yaml file: schema.yaml file loaded successfully]
[2025-02-11 16:20:42,266: INFO: common: created directory at artifacts]
[2025-02-11 16:20:42,266: INFO: common: created directory at artifacts/model_evaluation]
[2025-02-11 16:20:42,727: INFO: common: json file created at artifacts\model_evaluation\metrics.json]


2025/02/11 16:20:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ELASTICNET'.
2025/02/11 16:21:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ELASTICNET, version 1
Created version '1' of model 'ELASTICNET'.


🏃 View run luxuriant-conch-1000 at: https://dagshub.com/bhuvankatakam/Data-Science-Project.mlflow/#/experiments/0/runs/c15ebded44cd470f82199a898b50cb9c
🧪 View experiment at: https://dagshub.com/bhuvankatakam/Data-Science-Project.mlflow/#/experiments/0
